<a href="https://colab.research.google.com/github/joepeskett/tree-pixels/blob/master/notebooks/TF_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using TF Datasets

There are some really good walkthroughs of how this can be achieved in the [tensorflow data api documentation](https://www.tensorflow.org/guide/data#consuming_text_data). 

Get the dataset from stanford

In [0]:
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar -xvf aclImdb_v1.tar.gz

In [0]:
import tensorflow_datasets as tfds
#dataset = tfds.load("imdb_reviews")

In [0]:
# We want to create a similar dataset to the one found in the the tfds package. 
# How we would we efficiently load these files to create a useful dataset

import tensorflow as tf



Notes about this data set - the rating for each movie is in the filename, as is the unique ID of the particular review. 

A good first step would be to have a setup to have a unique ID, text and the rating. 

TODO:

* List the files within the positive and negative training directories. 
* Load these into an appropriate training data set
* Ensure each piece is text is labelled correctly
* Shuffle these dataset
* Create an appropriate representation for the text data, likely using the TextVectorisation layer within Keras
* Add an embedding layer
* Train a model 
* Try to optimise the pipeline that we've prepared

In [0]:
import os
DIRS = ["aclImdb/train/pos/", "aclImdb/train/neg/"]
files = []
for d in DIRS:
  for f in os.listdir(d):
    files.append(os.path.join(d, f))

In [0]:
# Simple function to grab the ID and rating from the file name
# We want to return tf.constants
def labeller(file_name):
  id, rating = os.path.splitext(os.path.basename(file_name))[0].split("_")
  return [tf.constant(id), tf.constant(rating)]

In [0]:
# create tf 
labeller(files[1])

In [0]:
file_path_dataset = tf.data.Dataset.list_files(files)

In [0]:
file_path_dataset

In [0]:
# Now we want to create a data set for each individual file

IDs = []
reviews = [] 
ratings = []
for filename in files[:10]:
  lines_dataset = tf.data.TextLineDataset(filenames=filename)
  id, rating = labeller(filename)
  IDs.append(id)
  ratings.append(rating)
  reviews.append(lines_dataset)


test_data = tf.data.Dataset.from_tensor_slices(tf.constant(IDs), tf.constant(reviews), tf.constant(ratings))

In [0]:
example_dataset = tf.data.TextLineDataset(os.path.join(files[1][0], files[1][1]))
example_dataset = example_dataset.map

In [0]:
print(example_dataset)